<a href="https://colab.research.google.com/github/Mishra-0709/Anveshan-Smart-Vehicle-BlackGriffin/blob/main/Lane_Detection_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install OpenCV if not already installed
!pip install -q opencv-python moviepy

# Import necessary libraries
import cv2
import numpy as np
from moviepy.editor import VideoFileClip

# Function to process video frames for lane detection
def process_video(input_video_path, output_video_path):
    """
    Processes an input video to detect lane lines and saves the output video.
    :param input_video_path: Location of the input video file
    :param output_video_path: Location to save the output video file with detected lane lines
    """
    input_video = VideoFileClip(input_video_path)
    processed_video = input_video.fl_image(process_frame)
    processed_video.write_videofile(output_video_path, audio=False)

# Frame processing function for lane detection
def process_frame(image):
    """
    Detect lane lines in an individual frame.
    :param image: A single frame (image) from the video
    :return: The frame with detected lane lines drawn on it
    """
    # Step 1: Grayscale conversion
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Step 2: Apply Gaussian Blur for noise reduction
    kernel_size = 5
    blur_gray = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)

    # Step 3: Canny Edge Detection
    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

    # Step 4: Region of Interest Selection
    region = region_of_interest(edges)

    # Step 5: Hough Transform to Detect Lane Lines
    lines = hough_lines(region)

    # Step 6: Draw Lane Lines on the Original Image
    result = draw_lane_lines(image, lane_lines(image, lines))
    return result

# Define the Region of Interest (ROI) Mask
def region_of_interest(image):
    """
    Applies an image mask to focus on the area where lane lines are expected.
    :param image: Edge-detected image
    :return: Masked image with only region of interest
    """
    mask = np.zeros_like(image)
    ignore_mask_color = 255
    rows, cols = image.shape[:2]
    bottom_left = [cols * 0.1, rows * 0.95]
    top_left = [cols * 0.4, rows * 0.6]
    bottom_right = [cols * 0.9, rows * 0.95]
    top_right = [cols * 0.6, rows * 0.6]
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

# Hough Transform for Line Detection
def hough_lines(image):
    """
    Applies Hough Transform to detect lines in the masked edge-detected image.
    :param image: Masked edge-detected image
    :return: Lines detected using Hough Transform
    """
    rho = 1            # distance resolution in pixels
    theta = np.pi / 180  # angular resolution in radians
    threshold = 20      # minimum number of votes
    min_line_length = 20 # minimum number of pixels to qualify as a line
    max_line_gap = 300   # maximum gap in pixels between line segments to link them
    lines = cv2.HoughLinesP(image, rho, theta, threshold, np.array([]),
                            minLineLength=min_line_length, maxLineGap=max_line_gap)
    return lines

# Functions to draw lane lines and calculate slope
def average_slope_intercept(lines):
    """
    Calculates the average slope and intercept for the left and right lanes.
    :param lines: Array of lines detected from Hough Transform
    :return: Left and right lane lines
    """
    left_lines = []
    right_lines = []
    left_weights = []
    right_weights = []

    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1) if x1 != x2 else 0
            intercept = y1 - slope * x1
            length = np.sqrt((y2 - y1)**2 + (x2 - x1)**2)

            if slope < 0:  # left lane
                left_lines.append((slope, intercept))
                left_weights.append((length))
            else:  # right lane
                right_lines.append((slope, intercept))
                right_weights.append((length))

    left_lane = np.dot(left_weights, left_lines) / np.sum(left_weights) if len(left_weights) > 0 else None
    right_lane = np.dot(right_weights, right_lines) / np.sum(right_weights) if len(right_weights) > 0 else None

    return left_lane, right_lane

def pixel_points(y1, y2, line):
    """
    Converts a line's slope and intercept to pixel points.
    :param y1, y2: The start and end y-coordinates of the line
    :param line: Slope and intercept of the line
    :return: (x1, y1), (x2, y2) points of the line
    """
    if line is None:
        return None
    slope, intercept = line
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    return (x1, y1), (x2, y2)

def lane_lines(image, lines):
    """
    Compute the coordinates for lane lines.
    :param image: The input frame
    :param lines: Lines detected from Hough Transform
    :return: Pixel points for the left and right lane lines
    """
    left_lane, right_lane = average_slope_intercept(lines)
    y1 = image.shape[0]
    y2 = int(y1 * 0.6)
    left_line = pixel_points(y1, y2, left_lane)
    right_line = pixel_points(y1, y2, right_lane)
    return left_line, right_line

# Draw lane lines on the image
def draw_lane_lines(image, lines, color=[255, 0, 0], thickness=10):
    """
    Draw the computed lane lines onto the frame.
    :param image: Input frame
    :param lines: Computed left and right lane lines
    :param color: Color of the lane lines
    :param thickness: Thickness of the lane lines
    :return: Frame with lane lines drawn
    """
    line_image = np.zeros_like(image)
    for line in lines:
        if line is not None:
            cv2.line(line_image, *line, color, thickness)
    return cv2.addWeighted(image, 1.0, line_image, 1.0, 0.0)

# Run lane detection on an example video
input_video_path = "input_video.mp4"  # Path to the input video
output_video_path = "output_video.mp4"  # Path to save the processed video
process_video(input_video_path, output_video_path)


  if event.key is 'enter':



OSError: MoviePy error: the file input_video.mp4 could not be found!
Please check that you entered the correct path.